<a href="https://colab.research.google.com/github/Arghya-Sengupta/Major-Project/blob/main/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# run this code to clone the yolov5 repository from github

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-74-gcf4f3c3 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.7/107.7 GB disk)


In [5]:
# if you are using google colab
# run this code to fetch data from google drive

from google.colab import drive
drive.mount('/content/drive')
!unzip -q /content/drive/MyDrive/Data/UECFOOD100.zip -d /content/
!cp "/content/drive/MyDrive/Data/custom_coco.yaml" "/content/yolov5/data/"
print("Check for /content/yolov5/data/custom_coco.yaml")

Check for /content/yolov5/data/custom_coco.yaml


In [6]:
# do with 60 epochs for best results

!python train.py --img 640 --batch 2 --epochs 3 --data /content/yolov5/data/custom_coco.yaml --weights yolov5s.pt --cache

# runs/train/exp/weights/best.pt
# yolov5x.pt

       0/2        0G    0.0875    0.0315         0         3       640:   6% 14/250 [00:38<10:47,  2.74s/it]
Traceback (most recent call last):
  File "train.py", line 330, in train
    pred = model(imgs)  # forward
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/yolov5/models/yolo.py", line 130, in forward
    return self._forward_once(x, profile, visualize)  # single-scale inference, train
  File "/content/yolov5/models/yolo.py", line 153, in _forward_once
    x = m(x)  # run
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/yolov5/models/common.py", line 275, in forward
    return torch.cat(x, self.d)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "train.py", line 643, in <module>
  

In [ ]:
# detect a custom image
#!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.45 --source /content/test1.jpg